### Determining the optimal number of hidden layers and neurons for an Artificial Neural Network (ANN) 
This can be challenging and often requires experimentation. However, there are some guidelines and methods that can help you in making an informed decision:

- Start Simple: Begin with a simple architecture and gradually increase complexity if needed.
- Grid Search/Random Search: Use grid search or random search to try different architectures.
- Cross-Validation: Use cross-validation to evaluate the performance of different architectures.
- Heuristics and Rules of Thumb: Some heuristics and empirical rules can provide starting points, such as:
  -    The number of neurons in the hidden layer should be between the size of the input layer and the size of the output layer.
  -  A common practice is to start with 1-2 hidden layers.

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import pickle

In [3]:
data=pd.read_csv('Churn_Modelling.csv')
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

onehot_encoder_geo = OneHotEncoder(handle_unknown='ignore')
geo_encoded = onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

data = pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)

X = data.drop('Exited', axis=1)
y = data['Exited']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Save encoders and scaler for later use
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo, file)

with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

In [4]:
## Define a function to create the model and try different parameters(KerasClassifier)

def create_model(neurons=32,layers=1):
    model=Sequential()
    model.add(Dense(neurons,activation='relu',input_shape=(X_train.shape[1],)))

    for _ in range(layers-1):
        model.add(Dense(neurons,activation='relu'))

    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='adam',loss="binary_crossentropy",metrics=['accuracy'])

    return model



In [5]:
## Create a Keras classifier
model=KerasClassifier(layers=1,neurons=32,build_fn=create_model,verbose=1)

In [6]:

# Define the grid search parameters
param_grid = {
    'neurons': [16, 32, 64, 128],
    'layers': [1, 2],
    'epochs': [50, 100]
}

In [7]:
# Perform grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3,verbose=1)
grid_result = grid.fit(X_train, y_train)

# Print the best parameters
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Fitting 3 folds for each of 16 candidates, totalling 48 fits


/opt/anaconda3/lib/python3.12/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/opt/anaconda3/lib/python3.12/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/opt/anaconda3/lib/python3.12/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/opt/anaconda3/lib/python3.12/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/opt/anaconda3/lib/python3.12/si

Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 2s 542us/step - accuracy: 0.6670 - loss: 0.6075
Epoch 2/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 2s 488us/step - accuracy: 0.7716 - loss: 0.6114
Epoch 2/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.7872 - loss: 0.5361
Epoch 2/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8048 - loss: 0.4451
Epoch 3/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7620 - loss: 0.5789
Epoch 2/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7792 - loss: 0.5253
Epoch 2/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.5116 - loss: 0.7241
Epoch 2/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.4181 - loss: 0.8058
Epoch 2/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.5616 - loss: 0.7159
 31/167 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8010 - loss: 0.4432  Epoch 2/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 883us/step - accuracy: 0.8

/opt/anaconda3/lib/python3.12/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/opt/anaconda3/lib/python3.12/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential model

167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 863us/step - accuracy: 0.8646 - loss: 0.3285
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/stepep - accuracy: 0.8624 - loss: 0.33
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8659 - loss: 0.3188
Epoch 50/50
141/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8609 - loss: 0.3334Epoch 1/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8614 - loss: 0.3324
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 750us/step - accuracy: 0.8659 - loss: 0.3131
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Epoch 1/50
Epoch 1/50


...

167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8712 - loss: 0.3027
Epoch 15/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 515us/step - accuracy: 0.8468 - loss: 0.3748
Epoch 4/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 563us/step - accuracy: 0.8637 - loss: 0.3315
Epoch 11/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8672 - loss: 0.3257  
Epoch 21/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8842 - loss: 0.2625
Epoch 96/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8719 - loss: 0.3155
Epoch 16/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 864us/step - accuracy: 0.8644 - loss: 0.3308
Epoch 18/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8485 - loss: 0.3602
Epoch 5/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8711 - loss: 0.326554 
Epoch 12/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8766 - loss: 0.3115
Epoch 22/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 667us/step - accuracy: 0.7849 - loss: 0.4805
Epoch 2/100
167/167 ━━━━━━━━━━━━━━

/opt/anaconda3/lib/python3.12/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 912us/step - accuracy: 0.8842 - loss: 0.2839
Epoch 23/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8757 - loss: 0.3075
Epoch 25/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8367 - loss: 0.3852
Epoch 3/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8702 - loss: 0.3162
Epoch 29/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8680 - loss: 0.3145
Epoch 12/100
Epoch 1/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8699 - loss: 0.3275 
Epoch 9/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8675 - loss: 0.3105
Epoch 19/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8610 - loss: 0.3544
Epoch 4/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8733 - loss: 0.3133
Epoch 13/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8718 - loss: 0.3031
Epoch 26/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8744 - loss: 0.2957
Epoch 24/100
167/167 ━━━━━━━━━━━━

/opt/anaconda3/lib/python3.12/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 284us/step - accuracy: 0.7047 - loss: 0.5667
Epoch 2/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 252us/step - accuracy: 0.8216 - loss: 0.4144
Epoch 3/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 263us/step - accuracy: 0.8383 - loss: 0.3898
Epoch 4/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 255us/step - accuracy: 0.8549 - loss: 0.3680
Epoch 5/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 260us/step - accuracy: 0.8473 - loss: 0.3638
Epoch 6/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 262us/step - accuracy: 0.8472 - loss: 0.3593
Epoch 7/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 275us/step - accuracy: 0.8541 - loss: 0.3490
Epoch 8/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 253us/step - accuracy: 0.8619 - loss: 0.3396
Epoch 9/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 257us/step - accuracy: 0.8519 - loss: 0.3493
Epoch 10/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 391us/step - accuracy: 0.8664 - loss: 0.3317
Epoch 11/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 260us/step - accuracy: 0.8597 - loss: 0.3384
Epoch 12/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 